In [1]:
import copy
import numpy as np
import time
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision

from timm.loss import LabelSmoothingCrossEntropy
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from torchvision import datasets, models
from torchvision import transforms as T

/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


trainset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

testset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [4]:
classes = trainset.classes
num_class = len(classes)

In [5]:
class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model1 = TinyModel()
model2 = TinyModel()
model3 = TinyModel()
models = [model1, model2, model3]
weights = [0.1, 0.5, 0.8]
threshold=0.5
epochs=3

In [7]:
pso_pop = 7
fa_pop = 3
dim = 3
w = 0.5
alpha_0 = 0.9
alpha_f = 0.1
df = 0.5
states = np.array([[1.3,0.8],
                   [1.2,1.2],
                   [0.6,1.5]])
maxiter = 10
fa_options = {'bmin':1, 'gamma':0.8, 'alpha':0.2}
search_bounds = (np.array([0] * dim), np.array([1] * dim))
k_vals = 0.8, 0.2

In [8]:
model1 = TinyModel().to(mps_device)
model2 = TinyModel().to(mps_device)
model3 = TinyModel().to(mps_device)
models = [model1, model2, model3]
weights = [0.1, 0.5, 0.8]
threshold=0.5
epochs=3

In [42]:
from botanica.transformer.model_selection import _get_model_output
output = _get_model_output(model1, trainloader, mps_device)
len(output)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:02<00:00, 336.31it/s]


60000

In [43]:
from botanica.transformer.model_selection import get_suite_outputs
suite_train_output = get_suite_outputs(models, testloader, mps_device)
len(suite_train_output)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 89.59it/s]


3

In [44]:
suite_val_output = get_suite_outputs(models, testloader, mps_device)
len(suite_val_output)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 404.66it/s]


3

In [45]:
from botanica.transformer.model_selection import get_ensemble_output
ens_train_out = get_ensemble_output(suite_train_output, weights)
ens_val_out = get_ensemble_output(suite_val_output, weights)
print(len(ens_train_out))
print(len(ens_val_out))

10000
10000


In [46]:
sftmx = torch.nn.Softmax(dim=1)

In [47]:
out1 = torch.Tensor([[1,.5,1]])
out2 = torch.Tensor([[1,1,1]])
print(sftmx(out1))
print(sftmx(out2))
weights = [1, 2]
test_out = get_ensemble_output([out1, out2], weights)
test_out

tensor([[0.3837, 0.2327, 0.3837]])
tensor([[0.3333, 0.3333, 0.3333]])


tensor([[1.0503, 0.8994, 1.0503]])

In [48]:
len(models)

3

In [49]:
from botanica.transformer.model_selection import get_model_suite
get_model_suite([0.3, 0.5, 0.8], models, 0.5)

[TinyModel(
   (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
   (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
   (fc1): Linear(in_features=256, out_features=120, bias=True)
   (fc2): Linear(in_features=120, out_features=84, bias=True)
   (fc3): Linear(in_features=84, out_features=10, bias=True)
 ),
 TinyModel(
   (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
   (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
   (fc1): Linear(in_features=256, out_features=120, bias=True)
   (fc2): Linear(in_features=120, out_features=84, bias=True)
   (fc3): Linear(in_features=84, out_features=10, bias=True)
 )]

In [50]:
from botanica.transformer.model_selection import fitness_wrapper_ensemble
num_models = len(suite_val_output)
fitness = fitness_wrapper_ensemble(ens_val_out, num_models, (0.8,0.2), mps_device, classes, testset)

Loss: 2.3026
top_k acc: 0.1000, 0.3005, 0.5000


In [51]:
print(fitness)

tensor(-0.1467, device='mps:0')


In [52]:
models = [model1, model2, model3]
weights = [1, 1, 1]
threshold=0.5
suite_train_output = get_suite_outputs(models, testloader, mps_device)
suite_val_output = get_suite_outputs(models, testloader, mps_device)

def swarm_wrapper(position):
    train_suite = get_model_suite(position, suite_train_output, threshold)
    val_suite = get_model_suite(position, suite_val_output, threshold)
    num_models = len(train_suite)
    train_out = get_ensemble_output(train_suite, weights)
    val_out = get_ensemble_output(val_suite, weights)
    fitness = fitness_wrapper_ensemble(val_out, num_models, (0.8,0.2), mps_device, classes, testset)
    return fitness

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 90.11it/s]


In [53]:
swarm_wrapper([0.1, 1, 0.1])

Loss: 2.3026
top_k acc: 0.1000, 0.2994, 0.5026


tensor(-0.2800, device='mps:0')

Testing tensor loading

In [54]:
torch.save(out1, "model_1_val_logits.pt")
torch.save(out2, "model_2_val_logits.pt")

In [55]:
from botanica.transformer.model_selection import load_output_from_dir

In [56]:
output_tensors = load_output_from_dir('.', 2)

In [57]:
output_tensors

tensor([[[1.0000, 0.5000, 1.0000]],

        [[1.0000, 1.0000, 1.0000]]])

In [58]:
output_tensors[0]

tensor([[1.0000, 0.5000, 1.0000]])

In [59]:
output_tensors[0]

tensor([[1.0000, 0.5000, 1.0000]])

In [60]:
output_tensors.shape

torch.Size([2, 1, 3])

In [61]:
outputs = torch.stack(suite_val_output)
outputs = torch.rand(outputs.shape).to(mps_device)
weights = [1, 1.1, 0.1]
def swarm_wrapper(position):
    val_suite = get_model_suite(position, outputs, threshold)
    num_models = len(val_suite)
    val_out = get_ensemble_output(val_suite, weights)
    fitness = fitness_wrapper_ensemble(val_out, num_models, (0.8,0.2), mps_device, classes, testset)
    return fitness

In [62]:
outputs.shape

torch.Size([3, 10000, 10])

In [63]:
swarm_wrapper([1,0,0])

Loss: 2.3030
top_k acc: 0.1012, 0.3008, 0.4969


tensor(-0.2810, device='mps:0')

In [64]:
swarm_wrapper([0,1,0])

Loss: 2.3034
top_k acc: 0.0973, 0.2925, 0.4860


tensor(-0.2778, device='mps:0')

In [65]:
swarm_wrapper([0,0,1])

Loss: 2.3027
top_k acc: 0.1012, 0.3028, 0.5020


tensor(-0.2810, device='mps:0')

In [66]:
outputs[0]

tensor([[0.4409, 0.1645, 0.5835,  ..., 0.3703, 0.2833, 0.3343],
        [0.0653, 0.6834, 0.0220,  ..., 0.5134, 0.0226, 0.6736],
        [0.9890, 0.8564, 0.4762,  ..., 0.1573, 0.2617, 0.9479],
        ...,
        [0.2680, 0.3523, 0.8094,  ..., 0.8389, 0.9117, 0.3135],
        [0.7880, 0.8718, 0.5842,  ..., 0.8909, 0.9925, 0.5608],
        [0.5190, 0.5504, 0.5137,  ..., 0.3137, 0.0161, 0.9751]],
       device='mps:0')

In [67]:
type(testset.targets)

list

In [68]:
torch.stack(suite_val_output).shape

torch.Size([3, 10000, 10])

In [69]:
len(testset)

10000

In [70]:
type(testset.targets) == torch.Tensor

False

In [71]:
type(testset.targets) == list

True

In [72]:
from torch import Tensor

In [73]:
type(testset.targets) == Tensor

False

In [74]:
testset.targets = testset.targets.tolist()

AttributeError: 'list' object has no attribute 'tolist'